In [69]:
#!pip install beautifulsoup4

In [23]:
from bs4 import BeautifulSoup

In [70]:
#!pip install lxml

In [71]:
#!pip install requests

In [26]:
import requests
import pandas as pd

Scraping the Data From Wiki

In [27]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
source= requests.get(url).text
soup= BeautifulSoup(source,'lxml')

In [28]:
#print(soup.prettify())

In [29]:
table = soup.find('table')
#print(table.prettify())

In [30]:
tableframe = pd.read_html(source)
#print(tableframe)

In [31]:
df = pd.DataFrame(tableframe[0])
df.columns=['Postcode','Borough','Neighbourhood']
df
df1=df.drop(0)

In [32]:
df1.head()

,Postcode,Borough,Neighbourhood
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park


# Cleaning Up Wrangled Data, remove null boroughs, replace null neighborhoods with boroughs.

In [33]:
df1=df1[df1.Borough != 'Not assigned']

df1.head(10)
df_new=pd.DataFrame(df1)
df2=df1.groupby("Postcode").agg(lambda x:','.join(set(x)))
df2=df2.reset_index()
df2.shape

(103, 3)

In [34]:
df2.loc[df2['Neighbourhood']=="Not assigned",'Neighbourhood']=df2.loc[df2['Neighbourhood']=="Not assigned",'Borough']

len(df2[df2['Neighbourhood'] == 'Not assigned'])

0

In [35]:
df2.shape

(103, 3)

## I have experienced so much difficulty with getting GeoCoder to work that I resorted to using the prepared CSV file

### So far my only insights are that there are an asburd number of missing values on the Wiki Page. 

In [68]:
#!wget -O /resources/data/GeoCode.csv http://cocl.us/Geospatial_data

In [37]:
geodf = pd.read_csv("GeoCode.csv")
geodf.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [38]:
df3 = df2.set_index('Postcode').join(geodf.set_index('Postal Code'))
#print(df3)

In [39]:
df3.shape

(103, 4)

### Still no new insights, but on to the mapping and exploration!

In [40]:
df3.loc[df3['Borough'] == 'Downtown Toronto']

,Borough,Neighbourhood,Latitude,Longitude
Postcode,,,,
M4W,Downtown Toronto,Rosedale,43.679563,-79.377529
M4X,Downtown Toronto,"St. James Town,Cabbagetown",43.667967,-79.367675
M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
M5A,Downtown Toronto,"Harbourfront,Regent Park",43.654260,-79.360636
M5B,Downtown Toronto,"Ryerson,Garden District",43.657162,-79.378937
M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
M5H,Downtown Toronto,"Adelaide,Richmond,King",43.650571,-79.384568


In [41]:
df3.loc[df3['Borough'] == 'West Toronto']

,Borough,Neighbourhood,Latitude,Longitude
Postcode,,,,
M6H,West Toronto,"Dovercourt Village,Dufferin",43.669005,-79.442259
M6J,West Toronto,"Little Portugal,Trinity",43.647927,-79.419750
M6K,West Toronto,"Exhibition Place,Parkdale Village,Brockton",43.636847,-79.428191
M6P,West Toronto,"High Park,The Junction South",43.661608,-79.464763
M6R,West Toronto,"Parkdale,Roncesvalles",43.648960,-79.456325
M6S,West Toronto,"Swansea,Runnymede",43.651571,-79.484450


In [42]:
!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim

WARNING conda.base.context:use_only_tar_bz2(632): Conda is constrained to only using the old .tar.bz2 file format because you have conda-build installed, and it is <3.18.3.  Update or remove conda-build to get smaller downloads and faster extractions.
Solving environment: failed

PackagesNotFoundError: The following packages are not available from current channels:

  - anaconda/linux-64::grpcio==1.16.1=py36hf8bcb03_1 -> openssl[version='>=1.1.1,<1.1.2.0a0']

Current channels:

  - https://conda.anaconda.org/conda-forge/linux-64
  - https://conda.anaconda.org/conda-forge/noarch
  - https://repo.anaconda.com/pkgs/main/linux-64
  - https://repo.anaconda.com/pkgs/main/noarch
  - https://repo.anaconda.com/pkgs/r/linux-64
  - https://repo.anaconda.com/pkgs/r/noarch

To search for alternate channels that may provide the conda package you're
looking for, navigate to

    https://anaconda.org

and use the search bar at the top of the page.




In [43]:
address = 'Toronto, Canada'

geolocator = Nominatim(user_agent='cn_explorer')
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


Importing folium and mapping the local area of Toronto

In [44]:
import folium

In [54]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Borough'], toronto_data['Neighbourhood']):
        label = '{}, {}'.format(neighborhood, borough)
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color='blue',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(map_toronto)  
    
map_toronto

# Getting Into Foursquare For Exploratory Data

In [48]:
toronto_data = df3[df3['Borough'].str.contains('Toronto')].reset_index(drop=True)
toronto_data.head()

,Borough,Neighbourhood,Latitude,Longitude
0,East Toronto,The Beaches,43.676357,-79.293031
1,East Toronto,"Riverdale,The Danforth West",43.679557,-79.352188
2,East Toronto,"The Beaches West,India Bazaar",43.668999,-79.315572
3,East Toronto,Studio District,43.659526,-79.340923
4,Central Toronto,Lawrence Park,43.728020,-79.388790


In [55]:
CLIENT_ID = 'YMOIZYBKFUKQXKLXSXTH1POHNCTHG3MLYLYRJB513MCL2545' # your Foursquare ID
CLIENT_SECRET = 'CJEI03S3U5WVZ5VMU44TXGCYTYT011NGQA0P510BHZJZARAH' # your Foursquare Secret
VERSION = '20180604'

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: YMOIZYBKFUKQXKLXSXTH1POHNCTHG3MLYLYRJB513MCL2545
CLIENT_SECRET:CJEI03S3U5WVZ5VMU44TXGCYTYT011NGQA0P510BHZJZARAH


In [56]:
LIMIT = 100
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID,
    CLIENT_SECRET,
    VERSION,
    latitude,
    longitude,
    radius,
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=YMOIZYBKFUKQXKLXSXTH1POHNCTHG3MLYLYRJB513MCL2545&client_secret=CJEI03S3U5WVZ5VMU44TXGCYTYT011NGQA0P510BHZJZARAH&v=20180604&ll=43.653963,-79.387207&radius=500&limit=100'

In [67]:
results = requests.get(url).json()
#results

In [58]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [63]:
from pandas.io.json import json_normalize

In [64]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Downtown Toronto,Neighborhood,43.653232,-79.385296
1,Japango,Sushi Restaurant,43.655268,-79.385165
2,Cafe Plenty,Café,43.654571,-79.389450
3,Textile Museum of Canada,Art Museum,43.654396,-79.386500
4,Poke Guys,Poke Place,43.654895,-79.385052


In [65]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

79 venues were returned by Foursquare.


In [66]:
map_venues = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, name, categories in zip(nearby_venues['lat'], nearby_venues['lng'], nearby_venues['name'], nearby_venues['categories']):
        label = '{}, {}'.format(name, categories)
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color='blue',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(map_venues)  
    
map_venues

# Findings thusfar-

### Not much of note, though we do know now that the centralized location of the latitude and longitude for Toronto seem to show that the denser collection of neighborhoods is around the south-central ocean region, and that the rest of the neighborhoods seem to be rather evenly dispersed in the immediate area. 
#### Comparing this with the map of local venues, we can see that just north of the cluster of neighborhoods is a majorly dense clustering of venues. From their basic shape even on the map we can tell this is likely some sort of heavily developed downtown area, a great place to do further examinations dependent on criteria. 